# Sample-based Krylov Quantum Diagonalization (SKQD)

## Fermi-Hubbard Model Implementation

This notebook implements SKQD for the Fermi-Hubbard model using:
- **Simple initial state** (GHZ-like state)
- **Sample-based diagonalization** using `qiskit_addon_sqd`
- **Bitstring measurement** and classical post-processing

In [ ]:
# Import required libraries
import zipfile
import requests
from io import BytesIO

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import warnings
import openfermion as of

from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.synthesis import LieTrotter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_ibm_runtime import SamplerV2 as Sampler

from qiskit_addon_sqd.counts import counts_to_arrays
from qiskit_addon_sqd.qubit import solve_qubit
from collections import Counter

warnings.filterwarnings("ignore")
print("✓ Libraries imported")

## 1. Load Fermi-Hubbard Hamiltonian from HamLib

In [ ]:
import hamlib.hamlib_snippets as hs
hdf5_file = "hamlib/condensedmatter/fermihubbard/FH_D-1.hdf5"
group_key = "/fh-graph-1D-grid-nonpbc-qubitnodes_Lx-6_U-12_enc-jw"

H_of = hs.read_openfermion_hdf5(
    fname_hdf5=hdf5_file,
    key=group_key,
    optype=of.QubitOperator
)

print(f"Loaded Hamiltonian with {len(H_of.terms)} Pauli terms")

In [ ]:
# Convert to Qiskit SparsePauliOp
def of_to_qiskit(op):
    """Convert OpenFermion QubitOperator to Qiskit SparsePauliOp"""
    n_qubits = max(q for term in op.terms for q, _ in term) + 1
    labels, coeffs = [], []
    for term, coeff in op.terms.items():
        pauli = ["I"] * n_qubits
        for q, p in term:
            pauli[q] = p
        labels.append("".join(pauli[::-1]))
        coeffs.append(coeff)
    return SparsePauliOp(labels, coeffs)

H_op = of_to_qiskit(H_of)
n_qubits = H_op.num_qubits
H_matrix = np.array(H_op.to_matrix())

# Compute exact ground state
eigenvalues, eigenvectors = np.linalg.eigh(H_matrix)
exact_gs_energy = eigenvalues[0]
ground_state = eigenvectors[:, 0]

print(f"System size: {n_qubits} qubits")
print(f"Hilbert space dimension: {2**n_qubits}")
print(f"Exact ground state energy: {exact_gs_energy:.8f}")

## 2. Prepare Initial State

In [ ]:
# Prepare the reference state for evolution
# Using a GHZ-like state (similar to skqd.ipynb)

qc_state_prep = QuantumCircuit(n_qubits)
qc_state_prep.h(0)
for i in range(n_qubits - 1):
    qc_state_prep.cx(i, i+1)

print(f"Initial State Preparation:")
print(f"  • State: GHZ-like (H on q0, then CNOT chain)")
print(f"  • Qubits: {qc_state_prep.num_qubits}")
print(f"  • Depth: {qc_state_prep.depth()}")
print(f"  • Gates: {qc_state_prep.count_ops()}")
qc_state_prep.draw("mpl", fold=-1)

## 3. Sample-based Krylov Quantum Diagonalization (SKQD)

Unlike the Hadamard test approach, SKQD:
1. Measures each evolved state $|\psi_k\rangle = U^k|\psi_0\rangle$ to obtain bitstrings
2. Accumulates samples from all states
3. Uses classical post-processing (`solve_qubit`) to diagonalize in the sampled basis

In [ ]:
# SKQD Configuration
krylov_dim = 2
dt = 0.3
num_trotter_steps = 12

print("="*70)
print("SAMPLE-BASED KQD CONFIGURATION (SKQD)")
print("="*70)
print(f"\n📋 Parameters:")
print(f"  • Time step (dt):        {dt}")
print(f"  • Trotter steps:         {num_trotter_steps}")
print(f"  • Krylov dimension:      {krylov_dim}")
print(f"  • Initial state:         GHZ-like state")
print(f"  • Method:                Sample-based (qiskit_addon_sqd)")

# Ground state energy for reference
gse = exact_gs_energy
print(f"\n  • Exact ground state:    {gse:.6f}")

In [ ]:
# Build time evolution circuits
evol_gate = PauliEvolutionGate(
    H_op, time=(dt / num_trotter_steps), synthesis=LieTrotter(reps=num_trotter_steps)
)  # U operator

qr = QuantumRegister(n_qubits)
qc_evol = QuantumCircuit(qr)
qc_evol.append(evol_gate, qargs=qr)

circuits = []
for rep in range(krylov_dim):
    circ = qc_state_prep.copy()
    
    # Repeating the U operator to implement U^0, U^1, U^2, ...
    for _ in range(rep):
        circ.compose(other=qc_evol, inplace=True)
    
    circ.measure_all()
    circuits.append(circ)

print(f"Built {len(circuits)} Krylov circuits (Lie-Trotter)")
print(f"Circuit depth for k=0: {circuits[0].depth()}")
print(f"Circuit depth for k=1: {circuits[1].depth()}")
print(f"Circuit depth for k={krylov_dim-1}: {circuits[-1].depth()}")

In [ ]:
# Visualize first few circuits
circuits[0].decompose().draw("mpl", fold=-1)

In [ ]:
circuits[1].decompose().draw("mpl", fold=-1)

In [ ]:
# Transpile circuits for backend
backend = GenericBackendV2(num_qubits=n_qubits+1)
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
isa_circuits = pm.run(circuits=circuits)

print(f"✓ Circuits transpiled for GenericBackendV2")

In [ ]:
# Run circuits with SamplerV2
sampler = Sampler(mode=backend)
job = sampler.run(isa_circuits, shots=1_000_000)  # Takes approximately 2-3 minutes

print(f"Running {len(isa_circuits)} circuits with 1,000,000 shots each...")
print("This may take a few minutes...")

In [ ]:
# Get counts from results
counts_all = [job.result()[k].data.meas.get_counts() for k in range(krylov_dim)]
print(f"✓ Measurement complete")

In [ ]:
# Accumulate counts cumulatively (as in the SKQD approach)
counts_cumulative = []
for i in range(krylov_dim):
    counter = Counter()
    for d in counts_all[: i + 1]:
        counter.update(d)
    
    counts = dict(counter)
    counts_cumulative.append(counts)

print(f"✓ Accumulated counts for {len(counts_cumulative)} Krylov dimensions")
for i in range(krylov_dim):
    print(f"  K={i+1}: {len(counts_cumulative[i])} unique bitstrings, {sum(counts_cumulative[i].values())} total counts")

In [ ]:
# Run SKQD using qiskit_addon_sqd
scipy_kwargs = {"k": 3, "which": "SA"}

ground_state_energies = []
print(f"\nRunning SKQD diagonalization...")

for idx, counts in enumerate(counts_cumulative):
    print(f"  K={idx+1}: {len(counts)} bitstrings, {sum(counts.values())} samples...", end=" ")
    
    # Convert counts to arrays
    bitstring_matrix, probs = counts_to_arrays(counts=counts)
    
    # Solve using sample-based quantum diagonalization
    eigenvals, eigenstates = solve_qubit(
        bitstring_matrix, H_op, verbose=False, **scipy_kwargs
    )
    
    gs_en = np.min(eigenvals)
    ground_state_energies.append(gs_en)
    
    print(f"E = {gs_en:.6f}, error = {abs(gs_en - gse):.6f}")

print(f"\n✓ SKQD complete!")

## 4. Results and Visualization

In [ ]:
# Final Results Summary
print("="*70)
print("FINAL RESULTS: SKQD FOR FERMI-HUBBARD")
print("="*70)

# Find best K
best_K = np.argmin([abs(e - gse) for e in ground_state_energies]) + 1
best_energy = ground_state_energies[best_K - 1]
best_error = abs(best_energy - gse)

print(f"\n🏆 RESULTS:")
print(f"  • Exact GS Energy:     {gse:.8f}")
print(f"  • Best SKQD (K={best_K}):     {best_energy:.8f}  (error: {best_error:.6f})")
print(f"  • Relative Error:      {best_error/abs(gse)*100:.4f}%")

print(f"\n📊 Energy per Krylov dimension:")
for i in range(krylov_dim):
    error = abs(ground_state_energies[i] - gse)
    print(f"  K = {i+1}: E = {ground_state_energies[i]:.8f}, error = {error:.6f}")

In [ ]:
# Plot results
fig, ax = plt.subplots(figsize=(10, 6))

# SKQD estimates
ax.plot(
    range(1, krylov_dim + 1),
    ground_state_energies,
    color="blue",
    linestyle="-.",
    marker="o",
    markersize=8,
    linewidth=2,
    label="SKQD Estimate",
)

# Exact ground state
ax.axhline(y=gse, color="red", linestyle="-", linewidth=2, label="Exact GS")

ax.set_xticks(range(1, krylov_dim + 1))
ax.set_xlabel("Krylov Space Dimension (K)", fontsize=12)
ax.set_ylabel("Energy", fontsize=12)
ax.set_title(
    "Sample-based Krylov Quantum Diagonalization (SKQD)\n4-site Fermi-Hubbard, U/t=12",
    fontsize=13
)
ax.legend(loc="upper right", fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Plot convergence (error vs K)
errors = [abs(e - gse) for e in ground_state_energies]

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(
    range(1, krylov_dim + 1),
    errors,
    color="blue",
    linestyle="-",
    marker="s",
    markersize=8,
    linewidth=2,
    label="SKQD Error",
)

ax.set_xticks(range(1, krylov_dim + 1))
ax.set_xlabel("Krylov Space Dimension (K)", fontsize=12)
ax.set_ylabel("Absolute Error", fontsize=12)
ax.set_title("SKQD Convergence", fontsize=13)
ax.legend(loc="upper right", fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Summary: Differences from exact ground state
print("\n" + "="*70)
print("DIFFERENCES (Exact - SKQD)")
print("="*70)

for i in range(krylov_dim):
    diff = gse - ground_state_energies[i]
    print(f"K = {i+1}: {diff:.6f}")

print("="*70)

*This code was part of the work done as part of the Qiskit Advocate Mentorship Programme (QAMP) 2025 project No.: 31.*\
*Mentors: Dr. Soham Pal, Dr. Shiplu Sarker,*\
*Mentees: Abdullah Afzal, Michael Papadopoulos, Gayathree M. Vinod.*\
*This notebook was prepared by Abdullah Afzal and verified by Michael Papadopoulos.*